In [13]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer                       
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Embedding, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import GlobalAveragePooling1D

from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
# use word embeddings to predict classes for labeled bbc aricles
# so articles will be passed into the ann as NxT 2d matrix
# N = num articles, T = num tokens (ie words per article)
# then articles get one hot encoded in the embedding layer, output NxTxD
# where D = vocab length

# but idk this isn't working yet

In [15]:
# all from tf_ann notebook

articles = []
labels = []

with open('input/bbc_text_cls.csv') as f:
    running_art = ''
    for l in f.readlines()[1:]:
        if '",' not in l or '", ' in l:
            running_art += f'{l.strip()} '
        else:
            last_line = l.split('",')
            assert len(last_line) == 2
            articles.append(running_art + last_line[0].strip())
            labels.append(last_line[1].strip())
            running_art = ''

label2int = {}
count = 0
for l in labels:
    if l not in label2int:
        label2int[l] = count
        count += 1

int_labels = [label2int[l] for l in labels]
df = pd.DataFrame(list(zip(articles, int_labels)), columns=['text', 'label'])

df.head()


,text,label
0,"""Ad sales boost Time Warner profit Quarterly ...",0
1,"""Dollar gains on Greenspan speech The dollar ...",0
2,"""Yukos unit buyer faces loan claim The owners...",0
3,"""High fuel prices hit BA's profits British Ai...",0
4,"""Pernod takeover talk lifts Domecq Shares in ...",0


In [16]:
# add tokenized text as new col

tokenizer = Tokenizer(
    num_words = 2000
)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences)

In [17]:
df['text_vec'] = data.tolist()
df.head()

,text,label,text_vec
0,"""Ad sales boost Time Warner profit Quarterly ...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"""Dollar gains on Greenspan speech The dollar ...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"""Yukos unit buyer faces loan claim The owners...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"""High fuel prices hit BA's profits British Ai...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"""Pernod takeover talk lifts Domecq Shares in ...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [18]:
df_train, df_test = train_test_split(df, test_size=0.3)
Xtrain = df_train['text_vec']
Xtest = df_test['text_vec']
Ytrain = df_train['label']
Ytest = df_test['label']

TypeError: one_hot() missing 1 required positional argument: 'n'

In [19]:
# num classes (K), counting from 1, equal to len of label2int
K = len(label2int)

# input feature len (D) equal to length of all features, in this case, the longest article (after padding, all articles are the same len)
# D = len(data[0])
# jk just choose arbitrary D embedding
D = 50

# T is the sequence len
T = len(data[0])

# V = the length of the word to int vector, aka vocab size
V = len(tokenizer.word_index)

K, D, V, T

(5, 50, 32359, 3488)

In [33]:
# Ytrain_oh = [one_hot(str(y), n=V) for y in Ytrain]
# Ytest_oh = [one_hot(str(y), n=V) for y in Ytest]

# len(Ytrain_oh), len(Ytrain)#.shape

(1506, 1506)

In [20]:
i = Input(shape=(T,))
x = Embedding(V, 20)(i)
x = Dense(20, activation='relu')(x)
x = Dense(K)(x)

model = Model(i, x)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 3488)]            0         
                                                                 
 embedding_3 (Embedding)     (None, 3488, 20)          647180    
                                                                 
 dense_6 (Dense)             (None, 3488, 20)          420       
                                                                 
 dense_7 (Dense)             (None, 3488, 5)           105       
                                                                 
Total params: 647,705
Trainable params: 647,705
Non-trainable params: 0
_________________________________________________________________


In [21]:
# i = Input(shape=(D,))
# x = Embedding(N, 30)(i)
# # GlobalAveragePooling1D()
# # x = Lambda(lambda t: tf.reduce_mean(t, axis=1))(x)
# x = Dense(30, activation='relu')(x)
# x = Dense(K)(x)

# model = Model(i, x)
# model.summary()

In [22]:
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)


In [23]:
if type(Xtrain) != list:
    Xtrain = Xtrain.tolist()
    Xtest = Xtest.tolist()
    Ytrain = Ytrain.tolist()
    Ytest = Ytest.tolist()


In [24]:
r = model.fit(
    Xtrain, Ytrain,
    validation_data=(Xtest, Ytest),
    epochs=10,
    batch_size=128
)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/Users/robbydivine/opt/miniconda3/envs/nlp/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/robbydivine/opt/miniconda3/envs/nlp/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/robbydivine/opt/miniconda3/envs/nlp/lib/python3.10/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/Users/robbydivine/opt/miniconda3/envs/nlp/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/robbydivine/opt/miniconda3/envs/nlp/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/Users/robbydivine

[1,
 1,
 2,
 3,
 1,
 4,
 0,
 3,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 1,
 0,
 3,
 2,
 1,
 3,
 1,
 1,
 3,
 3,
 2,
 1,
 0,
 2,
 1,
 1,
 2,
 3,
 1,
 1,
 3,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 4,
 4,
 4,
 2,
 2,
 1,
 4,
 1,
 1,
 1,
 0,
 2,
 0,
 2,
 2,
 3,
 3,
 1,
 4,
 3,
 0,
 1,
 0,
 1,
 2,
 1,
 2,
 3,
 4,
 2,
 0,
 1,
 3,
 4,
 3,
 4,
 4,
 0,
 4,
 1,
 4,
 2,
 4,
 4,
 0,
 0,
 2,
 1,
 0,
 0,
 3,
 2,
 1,
 4,
 1,
 3,
 0,
 1,
 3,
 1,
 1,
 3,
 4,
 4,
 4,
 3,
 1,
 3,
 3,
 0,
 0,
 0,
 4,
 3,
 0,
 0,
 0,
 1,
 2,
 0,
 4,
 0,
 1,
 2,
 0,
 2,
 0,
 2,
 3,
 3,
 0,
 4,
 1,
 2,
 4,
 0,
 3,
 2,
 2,
 2,
 2,
 3,
 0,
 2,
 3,
 4,
 2,
 1,
 1,
 2,
 3,
 0,
 3,
 4,
 0,
 2,
 2,
 3,
 4,
 3,
 3,
 0,
 2,
 4,
 4,
 0,
 4,
 3,
 0,
 3,
 0,
 0,
 1,
 0,
 4,
 0,
 0,
 3,
 1,
 4,
 1,
 1,
 1,
 3,
 0,
 4,
 3,
 2,
 3,
 2,
 4,
 4,
 3,
 4,
 1,
 2,
 1,
 3,
 2,
 2,
 3,
 2,
 2,
 1,
 1,
 3,
 1,
 3,
 2,
 0,
 4,
 1,
 2,
 2,
 0,
 0,
 4,
 4,
 0,
 0,
 3,
 3,
 4,
 2,
 3,
 3,
 2,
 1,
 3,
 2,
 2,
 4,
 3,
 0,
 3,
 3,
 4,
 0,
 0,
 3,
 0,
 3,
 0,
 3,
 2,
 4,
 4,


In [4]:
# somewhat straightforward stuff below around Tokenizing and padding in tf.keras

tests = [
    'I love Teddy so much.',
    'I am worried when Teddy gets diarrhea.',
    'I hope I can sleep soon.'
]

In [5]:
max_vocab_size = 20000
tokenizer = Tokenizer(
    num_words = max_vocab_size
)
tokenizer.fit_on_texts(tests)
sequences = tokenizer.texts_to_sequences(tests)
sequences

# tokenizer.fit_on_text(Xtrain)
# Xtrain = tokenizer.text_to_sequences(Xtrain)
# Xtrain = tokenizer.text_to_sequences(Xtest)

# tokenizer.texts_to_matrix(text) can convert to count or tfidf matrices

[[1, 3, 2, 4, 5], [1, 6, 7, 8, 2, 9, 10], [1, 11, 1, 12, 13, 14]]

In [7]:
tokenizer.word_index

{'i': 1,
 'teddy': 2,
 'love': 3,
 'so': 4,
 'much': 5,
 'am': 6,
 'worried': 7,
 'when': 8,
 'gets': 9,
 'diarrhea': 10,
 'hope': 11,
 'can': 12,
 'sleep': 13,
 'soon': 14}

In [8]:
# prepend 0s to ensure that all features are same length
data = pad_sequences(sequences) # optional args: maxlen=int, padding='post'
data

array([[ 0,  0,  1,  3,  2,  4,  5],
       [ 1,  6,  7,  8,  2,  9, 10],
       [ 0,  1, 11,  1, 12, 13, 14]], dtype=int32)